In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 150

In [48]:
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact

# Data
from transat.data import HYPOTHETICAL_SUBMISSION_DATE
from transat.data.load import download_historical, load_historical
from transat.data.split import split_historical
from transat.data.transform import preprocess_historical_basic, dataframe_to_array

# Metric
from transat.metric import mae

# Scenario/Simulation
from transat.data.scenario import generate_scenario

In [4]:
download_historical()
df = load_historical()

In [5]:
df = preprocess_historical_basic(df)

In [6]:
print("Spliting at : ", HYPOTHETICAL_SUBMISSION_DATE)
df_train, df_test = split_historical(df, HYPOTHETICAL_SUBMISSION_DATE)

Spliting at :  2020-07-31


In [33]:
nb_lookback_days = 30
sequence_format = True

(X_train, y_train), (X_cols, y_col) = dataframe_to_array(df_train, nb_lookback_days=nb_lookback_days,
    sequence_format=sequence_format)
(X_test, y_test), _ = dataframe_to_array(df_test, nb_lookback_days=nb_lookback_days,
    sequence_format=sequence_format)

X_train, y_train = X_train.reshape(X_train.shape[0], -1), y_train.reshape(-1)
X_test, y_test = X_test.reshape(X_test.shape[0], -1), y_test.reshape(-1)

print("X_train shape: ", np.shape(X_train))
print("y_train shape: ", np.shape(y_train))
print()
print("X_test  shape: ", np.shape(X_test))
print("y_test  shape: ", np.shape(y_test))

X_train shape:  (48412, 390)
y_train shape:  (48412,)

X_test  shape:  (27930, 390)
y_test  shape:  (27930,)


In [34]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.

from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

# from sklearn.utils import shuffle
# X_train, y_train = shuffle(X_train, y_train)

# Split data into train and test sets
X_train, X_valid , y_train, y_valid= train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    random_state=301
)

model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=20000,
              positive=True,
              selection='random')
# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, max_iter=20000, positive=True, precompute=True,
      selection='random')

In [35]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

valid_preds = model.predict(X_valid)
valid_preds = np.maximum(valid_preds, 0) # Don't predict negative cases
print('Valid MAE:', mae(valid_preds, y_valid))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 144.76205640510753
Valid MAE: 144.08718562729703
Test MAE: 593.6835222049625


In [50]:
np.shape(train_preds)

(38729,)

In [36]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + X_cols[0])
for d in range(-nb_lookback_days, 1):
    for col_name in X_cols[1:]:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -28 C5_Close public transport 7.393085491894867
Day -24 C5_Close public transport 5.096308071819734
Day -23 C6_Stay at home requirements 7.8323086418297265
Day -8 C6_Stay at home requirements 0.04439534791377654
Day -7 C7_Restrictions on internal movement 0.3867433709844503
Day -6 C8_International travel controls 0.2551910499458127
Day -5 H1_Public information campaigns 0.060916798635437884
Day -4 H2_Testing policy 0.03428772542445764
Day -3 H3_Contact tracing 0.09731438685625388
Day -2 H6_Facial Coverings 0.1919230738622441
Intercept 29.19191189624172


(1, 30, 13)
(1, 30, 1)


In [42]:
def simulate_scenario(X_scenario, y_scenario):
    # Simulate scenario

    X_sim = X_scenario.copy()
    X_sim_cases = X_sim[:,:,:1]
    X_sim_npis = X_sim[:,:,1:]
    y_sim = np.zeros(np.shape(y_scenario))

    nb_lookback_days = X_sim.shape[1]

    for d in range(y_sim.shape[1]):

        y = model.predict(X_sim.reshape(1,-1))
        y_sim[0,d,0] = max(y[0], 0)

        # Assuming constant NPIs here
        X_sim_npis = np.concatenate([X_sim_npis[:,1:], X_sim_npis[:,-1:]], axis=1)
        X_sim_cases = np.concatenate([X_sim_cases[:,1:], y.reshape(-1, 1, 1)], axis=1)

        X_sim =  np.concatenate([X_sim_cases, X_sim_npis], axis=-1)
        X_sim = np.array(X_sim)
    
    return y_sim

In [44]:
def viz_scenario(geo_id, X_scenario, y_scenario, y_sim):
    mae_error = mae(y_scenario, y_sim)

    plt.figure()
    plt.title(geo_id)

    plot_input_x = np.arange(X_scenario.shape[1])
    plot_input_y = X_scenario[:,:,:1].reshape(-1)

    plt.plot(plot_input_x, plot_input_y, label="Input Scenario")

    plot_output_x = np.arange(y_scenario.shape[1])+X_scenario.shape[1]
    plot_output_x = np.concatenate([plot_input_x[-1:], plot_output_x])
    plot_output_y = np.concatenate([plot_input_y[-1:], y_scenario.reshape(-1)])
    plt.plot(plot_output_x, plot_output_y, label="Output Scenario")


    plot_output_y = np.concatenate([plot_input_y[-1:], y_sim.reshape(-1)])
    plt.plot(plot_output_x, plot_output_y, label="Output Simulation")

    plt.ylabel("New Cases")
    plt.xlabel("Days")
    ax = plt.gca()
    plt.text(0.3, 0.5, f"$MAE={mae_error:.2f}$", transform=ax.transAxes)
    plt.legend()
    plt.show()

In [49]:
def interactive_scenario(geo_id):
    nb_future_days=30

    X_scenario , y_scenario = generate_scenario(df_train, df_test, geo_id, nb_lookback_days=nb_lookback_days,
        nb_future_days=nb_future_days, sequence_format=sequence_format)

    y_sim = simulate_scenario(X_scenario, y_scenario)

    viz_scenario(geo_id, X_scenario, y_scenario, y_sim)
    
geo_ids = sorted(df.GeoID.unique())

w_geo_id = widgets.Dropdown(
    options=geo_ids,
    value='France__nan',
    description='GeoID:',
    disabled=False
)

interact(interactive_scenario, geo_id=w_geo_id)

interactive(children=(Dropdown(description='GeoID:', index=87, options=('Afghanistan__nan', 'Albania__nan', 'A…

<function __main__.interactive_scenario(geo_id)>